## Welcome to ZKBP 
- A (rust-based) Python module supporting mathematical functionalities needed for Zero-Knowledge Proof (ZKP) described in PADL. 

The tutorial is split into sections listed below:
1) Common Functionality 
2) Cryptographic Building Block
3) Usecase Walkthrough

Pre-requisites:
1) Python
2) ZKPB module listed under the python search path.
3) Rust Compilation Tools, rustup (if ZKPB module is not compiled)

***

## Common Functionality

Let's start by importing some basic functionalities.

In [ ]:
import zkbp
import os,sys
from pathlib import Path
path = os.path.realpath('1-zkbp-tutorial.ipynb')
sys.path.append(str(Path(path).parents[1]))  # go up 2 levels [1] to '/pyledger/'
from pyledger.zkutils import Commit,r_blend, Token
# Some Helper Variable for assertion
zero_string = "000000000000000000000000000000000000000000000000000000000000000000"



<sub>Note that multiplicative notation is used instead of additive one for group operation to be consistent with the whitepaper. However, function is expressed usign additive notation because of implementation detail (usage of EC as group).</sub>

Generators Generation:
- ZKBP make use of two distinct generators $g$ and $h \in \mathbb{G}$ which is generated using: `gh = zkbp.gen_GH()`.

Key Pair Generation:
- `key_pair = zkbp.gen_pb_sk(gh)` generates key in the form of $pk = h^{sk}$.
- `pk = key_pair.get_pk()` and `sk = key_pair.get_sk()` to unpack key pair into public key and secret key respectively.
- With sk, it is possible to regenrate the corresponding key_pair{pk,sk} using `zkbp.regen_pb_sk(gh,sk)`.

Random Scalar Element:
- `r=zkbp.gen_r()` generates a random scalar point, $r \xleftarrow[\text{}]{\text{\$}} \mathbb{G}$ whereby $\mathbb{G}$ is the scalar field. 
- `r=r_blend()` is needed to access native math functions such as `+,-, * `, for example negating can now be done with `rd = -r`.

Parsing Scalar Element (such as SK):
- `zkbp.to_scalar_from_str(key_pair.get_sk())` parses a string representation of scalar to scalar object.

Group Operations, $h^x, g^r, g^{x'}, p^x$ where g,h are generators, p is commit, x' is i32 and x is normal scalar element.
- `zkbp.h_to_x(gh, x)` computes $h^x$.
- `zkbp.g_to_r(gh, r)` computes $g^r$.
- `zkbp.g_to_x(gh, x)` computes $g^x$ whereby x $\in$ i32.
- `zkbp.p_to_x(commit,x)` computes $com^x$.

In [ ]:
# generate g and h
gh = zkbp.gen_GH()

# generate public and secret key
key_pair = zkbp.gen_pb_sk(gh)
pk = key_pair.get_pk()
sk = key_pair.get_sk()
sk_scalar_obj  = zkbp.to_scalar_from_str(key_pair.get_sk())
recons_key_pair = zkbp.regen_pb_sk(gh,sk_scalar_obj)
assert recons_key_pair.get_pk() == pk and recons_key_pair.get_sk()==sk, "Reconstructed PK or SK does not agree"

# generate a random scalar r
r = zkbp.gen_r()
# print(f'r is {r.get}')

# This r does not have add,mult,neg functions, so we use class r_blend()
r = r_blend()
rd = -r 
#print(f'r_blend is {r.to_str()} and rd+r is {(rd+r).to_str()}')
#print(f'minus r is {rd.to_str()} and length is {len(rd.to_str())}')


# Group Operations
hx = zkbp.h_to_x(gh, r.val)
gr = zkbp.g_to_r(gh, r.val)
gx = zkbp.g_to_x(gh, 123)

value = 13
r = zkbp.gen_r()
cm = zkbp.commit(value, r, gh)
comx = zkbp.p_to_x(cm, r)

***

## Cryptographic Building Block

Pedersen commitment,  $c = g^vh^r$
- `r = zkbp.gen_r()` and `value = 13` generates a value 13 with blinding factor $r$.
- `cm = zkbp.commit(value,r, gh)` to commit value 13, where $g^{13}h^{r}$.
- Alternatively, `r = r_blend()` and `cmd = Commit(gh,value,r)` also generate a commitment.
- `zkbp.add(cm,cm)` can be used to add two pedersen commitments, while `zkbp.sub(cm,cm)` subtract one from the other.
- ZKBP also provides the capability to add two stringified commit together via `zkbp.add_value_commits(commit_l,commit_r)` and subtraction via `zkbp.sub_value_commits(commit_l,commit_r)`.
- CommitmentPoint Deserialization from String can be done using `zkbp.from_str(comm_str_repr)`.

In [ ]:
# Pedersen commitment generation
import zkbp.zkbp

value = 13
r = zkbp.gen_r()
cm = zkbp.commit(value, r, gh)

# Commitment operations (Add, Sub)
cm_sample = zkbp.commit(value+value, r.sum(r), gh)
new_comm = zkbp.add(cm,cm)
assert new_comm.get == cm_sample.get, "commit3 = Add(commit1,commit2) should satisfies commit3 === Commit(commit1.value + commit2.value, commit1.r + commit2.r)"

new_comm = zkbp.sub(cm,cm)
assert new_comm.get == zero_string, "Sub(commit1,commit2) should return 0 if commit1 === commit2"

# Commitment operations for commitment string, returning the same string structure
new_comm_str_repr = zkbp.add_value_commits(cm.get,cm.get)
assert new_comm_str_repr == cm_sample.get, "commit3 = Add(commit1,commit2) should satisfies commit3 === Commit(commit1.value + commit2.value, commit1.r + commit2.r)"

new_comm_str_repr = zkbp.sub_value_commits(cm.get,cm.get)
assert new_comm_str_repr == zero_string, "Sub(commit1,commit2) should return 0 if commit1 === commit2"

# Construct a commit point from its string representation
# new_cm := sub(cm,cm) should be that new_cm === from_str(new_cm_str_repr)
new_comm = zkbp.sub(cm,cm)
new_comm_str_repr = zkbp.sub_value_commits(cm.get,cm.get)
new_cm = zkbp.from_str(new_comm_str_repr)
assert new_comm.get == new_cm.get, "Converting from commitment string representation should remain the same."


Token, $tk = h^{sk \cdot r}$
- `tk = key_pair.to_token(r.val)` generate token of the form $tk = h^{sk \cdot r}$.
- `tk = to_token_from_str(tkn_str_repr)` generate token from token string representation.
- `zkbp.print_token(tk)` to print token. Note that the stdout used by RUST might not be redirected correctly in jupyternotebook.
- `zkbp.add_token(tk1,tk2)` can be used to add two token, while `zkbp.sub_token(tk1,tk2)` subtract one from the other.
- `zkbp.to_token_from_pk(pk, scalar_r)` can also be used to generate token.

Commit - Token Operations
- ZKBP also provides the capability to add commit and token together via `zkbp.add_commit_token(commit,token)` and subtraction via `zkbp.sub_commit_token(commit,token)`.


In [ ]:
# Alternative option (Note that this is different object, Commit is python Commit-helper_class while zkbp.commit return a rust commit)
r = r_blend()
cmd = Commit(gh, value, r)
print(cmd.eval.get)

# we can also create a token that is just the public key raised to r
r = r_blend()
tk = key_pair.to_token(r.val)
print(tk.get)

# Token reconstruction
tk_recons = zkbp.to_token_from_str(tk.get)
assert tk.get == tk_recons.get, "Reconstructed token should not differ in value."

# Printing: Might not print out token in the notebook. (Will only print out ok)
print(zkbp.print_token(tk))

# Adding and Subtraction
r = r_blend()
tk1 = key_pair.to_token(r.val)
r2 = r_blend()
tk2 = key_pair.to_token(r2.val)
tk_ref = key_pair.to_token(r.val.sum(r2.val))
tk_sum = zkbp.add_token(tk1,tk2)
assert tk_sum.get == tk_ref.get, "Adding token should be the same as adding the blinding factor before constructing token"
tk_ref = key_pair.to_token(r.val.sum(r2.val.neg()))
tk_sum = zkbp.sub_token(tk1,tk2)
assert tk_sum.get == tk_ref.get, "Subtracting token should be the same as subtracting the blinding factor before constructing token"

# PK to token
ref_token = key_pair.to_token(r.val)
token = zkbp.to_token_from_pk(key_pair.get_pk(), r.val)
assert token.get == ref_token.get, "Tokens should be equal."

#Commit - token combined operation
value = 13
r = zkbp.gen_r()
cm = zkbp.commit(value, r, gh)
r = r_blend()
tk1 = key_pair.to_token(r.val)
element1 = zkbp.sub_commit_token(cm, tk1)
element2 = zkbp.add_commit_token(element1, tk1)
assert cm.get == element2.get, "Subtracting and then adding back the same element should be behave like an identity function."

____

## PoK ZKP Showcase

### ZKProof of Knowledge of dlog
- Let's start with some arbitrary value `sval = 10`, sample randomness `r = r_blend()`.
- Commit to the chosen value with `cm = Commit(gh,sval,r)` and generate a token with `tk = key_pair.to_token(r.val)`.
- Let's assume we want to prove the knowledge of $sk$ in the token $h^{sk \cdot r}$. 
- Start by calculating a new generator base with $\frac{cm}{g^{sval}} = g^{sval-sval}h^{r} = h^r$.
- Perform operation $\frac{cm}{g^{sval}}$ using  `h2r = zkbp.sub(zkbp.from_str(cm.eval.get), zkbp.from_str(zkbp.g_to_x(gh, sval).get))`.
- Now we can generate proof of knowledge of $sk$ in $h^{r \cdot sk}$ with $h^r$ as the generator.
- Generate proof using `pr = zkbp.sigma_dlog_proof_explicit_sha256(tk,key_pair,h2r)`.
- Verify proof using `t = zkbp.sigma_dlog_proof_explicit_verify_sha256(pr,h2r,tk)`.
- Note that variant without sha256 suffix uses sha512.
<!-- Token, tk is not used in sigma_dlog_proof_explicit_sha256, zkbp.sigma_dlog_proof_explicit_sha256(.. ,key_pair,h2r) where key_pair.sk is the witness and h2r is the generator-->

In [ ]:
# proof of knowledge: lets create a zk proof and then verify it

#g2val = zkbp.g_to_x(gh,value)
sval = 10
r = r_blend()
cm = Commit(gh,sval,r)
tk = key_pair.to_token(r.val)
#h2r = Commit(gh,0,r).eval
#tk = key_pair.to_token(r.val)
h2r = zkbp.sub(zkbp.from_str(cm.eval.get), zkbp.from_str(zkbp.g_to_x(gh, sval).get))
pr = zkbp.sigma_dlog_proof_explicit_sha256(tk,key_pair,h2r)
t = zkbp.sigma_dlog_proof_explicit_verify_sha256(pr,h2r,tk)

# print(pr)
print("Proof of Knowledge for DLog: ", t)
assert t, "Proof not verified."
# hash order: pk_t_rand_commitment, h2r, pk

### ZKProof of Knowledge of DLOG equivalent
- Let's start with some arbitrary value `sval = 10`, sample randomness `r = r_blend()`.
- Commit to the chosen value with `cm = Commit(gh,sval,r)` and generate a token with `tk = key_pair.to_token(r.val)`.
- Let's assume we want to prove the equivalent of sk in token_sum and pk
- Note that `sigma_eq_dlog_proof_sha256` requires token_sum/sk, therefore we also supply h_sum_r (that is token without sk) at the end.
- In short, we do `zkbp.sigma_eq_dlog_proof_sha256(tk, key_pair, gh, h_sum_r)` to compute the proof.
- `zkbp.sigma_eq_dlog_verify_sha256(pr, gh, h_sum_r, tk)` to verify the proof.

In [ ]:

r = r_blend()
val = 10
tk = key_pair.to_token(r.val)
cm = Commit(gh,val,r).eval

h_sum_r = zkbp.sub(cm, zkbp.g_to_x(gh, val)) # g^vh^r / g^v = h^r
pr = zkbp.sigma_eq_dlog_proof_sha256(tk, key_pair, gh, h_sum_r) #tk = h^{sk . r}, h_sum_r = h^r
# cm = Commit(gh,val,r).eval
# print(pr)
res = zkbp.sigma_eq_dlog_verify_sha256(pr, gh, h_sum_r, tk)
print("Proof of DLOG-Equiv: ",res)
assert res, "Proof of DLOG-Equiv Failed."

### ZKProof of Knowledge of Consistency

- Proof of consistency assert that for a given commitment and token, it is of the form of $\exists v,r : com = g^vh^r \wedge tk = pk^r$.
- `p= zkbp.consistency_proof(v,r,gh=(g,h), cm, tk, pk)` computes the proof.
- `zkbp.consistency_proof_verify(p, gh=(g,h), cm, tk, pk)` verifies the proof.

In [ ]:
r = r_blend()
rd = -r 
newr = rd + r
#print(newr.to_str())
v = 0
tk = key_pair.to_token(r.val)
# print(f'tk is {tk.get}')
cm = Commit(gh,v,r).eval
# print(f'cm is {cm.get}')
# print(f'pk is {pk}')
pr = zkbp.consistency_proof(v,r.val,gh,cm,tk,pk)
# print(pr)
result = zkbp.consistency_proof_verify(proof=pr, gh=gh, ped_cm=cm, token=tk,
                                               pubkey=key_pair.get_pk())
print("Proof of Consistency:", result)
assert(result)

***

## Range Proof 

-  `zkbp.range_proof_single(n_bit, val=v, gh=gh, r=r.val)` proves that 0 < v < $2^n$.
- `zkbp.range_proof_single_verify(pr, n_bit, gh, cm.eval)` verifies the proof $:g^v h^r \wedge 0 < v < 2^n$.

In [ ]:
# lets create a range proof
v = 255 #(try -1, 1 )
r = r_blend()
gh = zkbp.gen_GH()
cm = Commit(gh, v, r)
n_bit = 8 #(try 1, 2)
pr = zkbp.range_proof_single(n_bit=n_bit, val=v, gh=gh, r=r.val)
res = zkbp.range_proof_single_verify(pr, n_bit, gh, cm.eval)
# print(pr)
print(res)
assert res, "Range proof not verified."

### DLog Calculation

- Given a token, $tk = pk^r$ and commitment, $cm = g^vh^r$, computes $v$.
- `zkbp.get_brut_v(cm,tk,gh,key_pair,max_range)` computes the aforementioned v by bruteforcing from $g^{v\cdot sk} = \frac{cm^{sk}}{tk}$.
<!-- 2^16 (65536) takes 8s on Intel Year-2020 Laptop-CPU  -->

In [ ]:
v = 100
r = zkbp.gen_r()
cm = zkbp.commit(v, r, gh)
tk = key_pair.to_token(r)
max_range = 1000000

computed_v = zkbp.get_brut_v(cm,tk,gh,key_pair,max_range)
assert v==computed_v, "Brute-forcing g^v should give the correct v."

### Hashing Commit with SHA256

- hash_test1(cm1), hash_test2(cm1,cm2), hash_test3(cm1,cm2,cm3) computes hashes with input of {1,2,3} commits respectively.

In [ ]:
v = 10
r = zkbp.gen_r()
cm = zkbp.commit(v, r, gh)

hash1 = zkbp.hash_test1(cm)
hash2 = zkbp.hash_test2(cm,cm)
hash3 = zkbp.hash_test3(cm,cm,cm)

<!-- # # Helper Function
# for attr in dir(r):
#     print("obj.%s = %r" % (attr, getattr(r, attr)))\ -->

END